
Pre-processing the dataset to feed the keras neural network for classifying questions (1) and answers (0)

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re
import numpy as np
import math
import random

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

messages = pd.read_csv('/content/drive/MyDrive/movie-corpus/movie_lines_normalized.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

Mounted at /content/drive


In [ ]:
messages.columns = ['msg_line', 'user_id', 'movie_id', 'msg']

In [ ]:
messages.head(10)

,msg_line,user_id,movie_id,msg
0,L1045,u0,m0,They do not!
1,L1044,u2,m0,They do to!
2,L985,u0,m0,I hope so.
3,L984,u2,m0,She okay?
4,L925,u0,m0,Let's go.
5,L924,u2,m0,Wow
6,L872,u0,m0,Okay -- you're gonna need to learn how to lie.
7,L871,u2,m0,No
8,L870,u0,m0,"""""""I'm kidding. You know how sometimes you jus..."
9,L869,u0,m0,Like my fear of wearing pastels?


Cleaningthe conversations

In [ ]:
#remove charactes
def remove_char(txt):
    return re.sub('[^0-9]','', txt)

In [ ]:
#leaving just the number of the index, so L872 changes to 872
messages['msg_line_clean'] = [remove_char(msg) for msg in messages['msg_line']]

In [ ]:
#change the column type to number
messages['msg_line_clean'] = pd.to_numeric(messages['msg_line_clean'])

In [ ]:
messages = messages.sort_values(by=['msg_line_clean'])

In [ ]:

#set the column as the index
messages = messages.set_index('msg_line_clean')

In [ ]:
messages.head(10)

,msg_line,user_id,movie_id,msg
msg_line_clean,,,,
49,L49,u0,m0,Did you change your hair?
50,L50,u3,m0,No.
51,L51,u0,m0,You might wanna think about it
59,L59,u9,m0,I missed you.
60,L60,u8,m0,It says here you exposed yourself to a group o...
61,L61,u9,m0,It was a bratwurst. I was eating lunch.
62,L62,u8,m0,With the teeth of your zipper?
63,L63,u7,m0,You the new guy?
64,L64,u2,m0,So they tell me...


Removing entities

In [ ]:
entities = pd.read_csv('/content/drive/MyDrive/movie-corpus/entity_list_complete.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

In [ ]:
entities.columns = ['ent', 'type']

In [ ]:
entities['ent'] = entities['ent'].str.lower()

In [ ]:
entities.head()

,ent,type
0,kinda,ORG
1,the dallas times herald,ORG
2,queen louisa,PERSON
3,a.m,GPE
4,cousin hop,PERSON


In [ ]:
entities['ent_len'] = [len(e) for e in entities['ent']]

In [ ]:
s = entities['ent_len'].sort_values(ascending=False).index

In [ ]:
entities = entities.reindex(s)

In [ ]:

entities = entities.reset_index(drop=True)

In [ ]:
entities.head()

,ent,type,ent_len
0,"""""""how can the bolshevik cause gain respect am...",WORK_OF_ART,237
1,"""""""the premier wishes to inform the government...",WORK_OF_ART,192
2,""""""" come tuesday twelve a.m. bingo these like-...",WORK_OF_ART,182
3,"""""""the suggestion of the president regarding t...",WORK_OF_ART,155
4,"""""""the management of boyd's takes pleasure in ...",WORK_OF_ART,146


In [ ]:
data = messages['msg']

In [ ]:
ent_list =  ['PERSON', 'ORG', 'NORP', 'FAC', 'GPE', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART', 'LAW', 'LANGUAGE']
#ent_list =  ['LANGUAGE']

ent = list()
for i in range(len(entities.index)):
    if entities['type'][i] in ent_list:
        ent.append(entities['ent'][i])

In [ ]:
ent = list(set(ent))
print(len(ent))
print(ent)

27848
['tsch i know', 'seconal', 'the czech republic', 'the red sox', 'oracle', 'hermaphroditic', 'oscar', 'stansfield norman', 'ford fairlane', 'percheron', 'bundist', 'texans', 'public domain', "it's a wonderful life", 'fallbrooks', 'meantime bob', "frank galvin's", 'columbus medical', 'cor', 'a rock island', 'jamie', 'cledus', 'colorada', 'magnesia', 'adc', '"""the younger james gang?"', 't.j lane', "mah darlin' luke", 'thy juliet', 'redcoats', 'eau de', 'good-bye scottie', 'gretel', 'john vincennes', 'nails', '"""you said """', 'cardiff giant', 'claire', 'veridian iii', 'walter kenton', 'l-l-lyndon', 'jean-claude', 'tin man', 'takagi', 'moke', 'lieutenant casselle', 'nurse', 'sportsphone', 'sting', 'savieer', 'koolaid', 'soranhas', 'look earl', 'he;s', 'la compania', 'jerry nathan', 'the nightmare juice', 'jesus christ johnny', 'little johnny', 'floris', 'the ten commandments', "zowie drew's", 'whereabouts', 'wurlitzer', 'janina godlewska', 'el-rage', '"""the odd couple?"', '"""leo

In [ ]:
def remove_entity(corpus):
    corpus = corpus.split(' ')
    corpus = [c for c in corpus if c not in list(dict.keys())]
    return ' '.join(corpus)

In [ ]:
def remove_entity(corpus):
    corpus = corpus.split(' ')
    # Use ent instead of dict to access the intended entity list.
    corpus = [c for c in corpus if c not in ent]
    return ' '.join(corpus)

In [ ]:
# Opening conversation sequence

In [ ]:
#read the file with the conversation sequence
conv_seq = pd.read_csv('/content/drive/MyDrive/movie-corpus/movie_conversations.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

In [ ]:
conv_seq.columns = ['user1_id', 'user2_id', 'movie_id', 'sequence']

In [ ]:
conv_seq.head(10)

,user1_id,user2_id,movie_id,sequence
0,u0,u2,m0,['L194' 'L195' 'L196' 'L197']
1,u0,u2,m0,['L198' 'L199']
2,u0,u2,m0,['L200' 'L201' 'L202' 'L203']
3,u0,u2,m0,['L204' 'L205' 'L206']
4,u0,u2,m0,['L207' 'L208']
5,u0,u2,m0,['L271' 'L272' 'L273' 'L274' 'L275']
6,u0,u2,m0,['L276' 'L277']
7,u0,u2,m0,['L280' 'L281']
8,u0,u2,m0,['L363' 'L364']
9,u0,u2,m0,['L365' 'L366']


Build conversation sequence

In [ ]:
# Build conversation sequence

In [ ]:
def split_conversation(txt):
    txt_alt = txt.split(' ')
    return txt_alt

In [ ]:
def seq_to_list(seq):
    return [remove_char(s) for s in seq]

In [ ]:
#initializing the msg_2 column
messages['msg_2'] = '-'

In [ ]:
def link_conversations(seq_list, df, filter1, filter2):
    i = 0
    while i in range(len(seq_list)):
        if i+1 < len(seq_list):
            next_msg = df.loc[int(seq_list[i+1]), filter1]
            df.at[int(seq_list[i]), filter2] = next_msg
        i+=1

In [ ]:

#link each message with its answer
for conv in conv_seq['sequence']:
    #split each sequence by space
    seq = split_conversation(conv)

    #remove the char L from the sequences
    txt_alt = [remove_char(s) for s in seq]

    #use the conversation sequence to build the target answer for each message
    link_conversations(txt_alt, messages, 'msg', 'msg_2')

In [ ]:
messages.head(30)

,msg_line,user_id,movie_id,msg,msg_2
msg_line_clean,,,,,
49,L49,u0,m0,Did you change your hair?,No.
50,L50,u3,m0,No.,You might wanna think about it
51,L51,u0,m0,You might wanna think about it,-
59,L59,u9,m0,I missed you.,It says here you exposed yourself to a group o...
60,L60,u8,m0,It says here you exposed yourself to a group o...,It was a bratwurst. I was eating lunch.
61,L61,u9,m0,It was a bratwurst. I was eating lunch.,With the teeth of your zipper?
62,L62,u8,m0,With the teeth of your zipper?,-
63,L63,u7,m0,You the new guy?,So they tell me...
64,L64,u2,m0,So they tell me...,C'mon. I'm supposed to give you the tour.


In [ ]:
# Pre processing the msg

In [ ]:
data = messages['msg']

In [ ]:

lemmatizer = WordNetLemmatizer()
def pre_processing_text(corpus):
    #remove html tags
    corpus = re.sub(r'<.*?>', '', str(corpus))

    #remove non-alphanumeric characters
    corpus = re.sub(r'[^a-z A-Z 0-9 \s]', '', str(corpus))

    #remove duplicated spaces
    corpus = re.sub(r' +', ' ', str(corpus))

    #capitalization
    corpus = corpus.lower()

    #tokenization
    corpus = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)

    #lammatization
    corpus = [lemmatizer.lemmatize(c) for c in corpus]

    #remove punctuation
    corpus = [t for t in corpus if t not in string.punctuation]

    #remove stopwords
    #it makes the model worst
    #stopwords_ = stopwords.words("english")
    #corpus = [t for t in corpus if t not in stopwords_]

    return ' '.join(corpus)

In [ ]:
%%time
data_pre_processed = [pre_processing_text(str(m)) for m in data]
data_pre_processed

CPU times: user 20.9 s, sys: 244 ms, total: 21.2 s
Wall time: 21.4 s


['did you change your hair',
 'no',
 'you might wanna think about it',
 'i missed you',
 'it say here you exposed yourself to a group of freshman girl',
 'it wa a bratwurst i wa eating lunch',
 'with the teeth of your zipper',
 'you the new guy',
 'so they tell me',
 'cmon im supposed to give you the tour',
 'so which dakota you from',
 'north actually howd you',
 'i wa kidding people actually live there',
 'yeah a couple were outnumbered by the cow though',
 'how many people were in your old school',
 'thirtytwo',
 'get out',
 'how many people go here',
 'couple thousand most of them evil',
 'that im used to',
 'yeah but these guy have never seen a horse they just jack off to clint eastwood',
 'that girl i',
 'you burn you pine you perish',
 'who is she',
 'bianca stratford sophomore dont even think about it',
 'why not',
 'i could start with your haircut but it doesnt matter shes not allowed to date until her older sister doe and thats an impossibility',
 'katarina stratford my my yo

In [ ]:
messages['msg_pre_processed'] = data_pre_processed

Checking and removing duplicated messages in msg (not used)

In [ ]:
data = messages['msg_pre_processed']

In [ ]:
# Removing nan msg origined by '' messages

#filling the nan messages with a string- not necessary
#messages = messages.fillna('UNKNOWN')
messages = messages.dropna()

In [ ]:
# Removing apostrophes (need for embedding) (not used)
# Filling '-' messages with a generic one

#return generic answer
def generic_answer(txt):
  asw_list = ['talk more about it',
              'can you explain it better?',
              'I need to think more about it',
              'maybe...'
              ]
  if txt == '-':
    return random.choice(asw_list)
  return txt

In [ ]:
#seting a generic answer to the messages without answer
messages['msg_2'] = [generic_answer(msg) for msg in messages['msg_2']]

Tagging the msg with classes

In [ ]:
def define_target(corpus):

    if '?' in corpus:
        return 1
    else:
        return 0

In [ ]:
data = messages['msg']

In [ ]:
messages['target'] = [define_target(m) for m in data]

In [ ]:
messages['target'] = messages['target'].astype(int)

Save data

In [ ]:
messages

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed,target
msg_line_clean,,,,,,,
49,L49,u0,m0,Did you change your hair?,No.,did you change your hair,1
50,L50,u3,m0,No.,You might wanna think about it,no,0
51,L51,u0,m0,You might wanna think about it,talk more about it,you might wanna think about it,0
59,L59,u9,m0,I missed you.,It says here you exposed yourself to a group o...,i missed you,0
60,L60,u8,m0,It says here you exposed yourself to a group o...,It was a bratwurst. I was eating lunch.,it say here you exposed yourself to a group of...,0
...,...,...,...,...,...,...,...
666522,L666522,u9034,m616,So far only their scouts. But we have had repo...,maybe...,so far only their scout but we have had report...,0
666546,L666546,u9027,m616,Splendid site Crealock splendil I want to esta...,Certainly Sin,splendid site crealock splendil i want to esta...,0
666547,L666547,u9029,m616,Certainly Sin,talk more about it,certainly sin,0


In [ ]:
messages.to_csv('/content/drive/MyDrive/movie-corpus/movie_lines_pre_processed_keras.tsv', index=False, sep='\t', header=False)